In [1]:
import sys
sys.path.insert(1, '../')

from river import stream,tree,metrics
from encoding import prefix_bin
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

import warnings

warnings.filterwarnings("ignore")
import xgboost as xgb

import datetime, time
import utils
import sliding_window

c:\python3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [3]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [4]:
'''
Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
and obtain feature matrix to transform future events
'''

# Experiment settings
dataset_label = 'bpic17'

# Invoke parameters for dataset
with open('../dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    performance_measure_type = 'ROCAUC'

    dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('../result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'xgb'
    if classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01, verbosity =0)
    elif classifier == 'rf':
        streaming_classifier = RandomForestClassifier(n_estimators = 100)

    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    casecount = 0
    rowcounter = 0
    running_case = 0
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    
    # Sliding window for training setting
    window_size = 100
    test_size = 30
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier, 0]
    
    dataset = stream.iter_csv(
            dataset_loc
            )
    totallength = len(list(dataset))


    dataset = stream.iter_csv(
                dataset_loc,
                target='outcome'
                )
    enctype = 'Index-base'
    
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0

    for x,y in dataset:
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(maximum_prefix)]=set()

                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    x = streaming_db[caseid][maximum_prefix-1]
                    training_windows.update_window(x)
                    update_test_cases(streaming_db[caseid], test_cases)

                    if training_windows.retraining_count == test_size:
                        x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                        for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                        feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values
                        y_training = [i.true_label for i in training_windows.container]
                        training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training)
                        training_models['prefix_%s'%(maximum_prefix)][1] +=1
                        training_windows.reset_retraining_count()

                        '''
                        Evaluate current model with test cases
                        '''
                        model_update_count = training_models['prefix_%s'%(maximum_prefix)][1]
                        prediction_result[maximum_prefix][model_update_count] = {}
                        y_truelist = []
                        y_predlist = []
                        for case in test_cases:
                            if len(case) >= maximum_prefix:
                                x = case[maximum_prefix-1]
                                model = training_models['prefix_%s'%(x.prefix_length)][0]
                                current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                                current_event = pd.Series(current_event).to_frame().T
                                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba(current_event)
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                        prediction_result[maximum_prefix][model_update_count]['y_true'] = y_truelist
                        prediction_result[maximum_prefix][model_update_count]['y_pred'] = y_predlist
                        if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                            cdhappend[maximum_prefix] = model_update_count
    for evaluation_method in ['Accuracy', 'F1', 'ROCAUC']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob_ = prediction_result[prefix][update]['y_pred']
            prediction_class = training_models['prefix_%s'%(maximum_prefix)][0].classes_
            if len(prediction_class) == 1: prediction_class = ['False', 'True']
            y_pred_prob = []
            for i in y_pred_prob_:
                y_pred_prob.append({prediction_class[0]: i[0][0], prediction_class[1]: i[0][1]})
            y_pred = [max(x, key=x.get) for x in y_pred_prob]
            update_count.append(update)

            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true, y_pred, average='macro')
        #             warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                y_pred_prob = np.array([[x['False'], x['True']] for x in y_pred_prob])
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:, 1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')
        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)

        if os.path.exists('../img/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('../img/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))
        if os.path.exists('../result/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('../result/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))


        with open('../result/%s/%s/Model updates/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('../img/%s/%s/Model updates/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()
        

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
'''
Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
and obtain feature matrix to transform future events
'''

# Experiment settings
dataset_label = 'bpic17'

# Invoke parameters for dataset
with open('../dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    performance_measure_type = 'Accuracy'

    dataset_loc = '../DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('../result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'xgb'
    if classifier == 'xgb':
        streaming_classifier = xgb.XGBClassifier(n_estimators = 100, learning_rate=0.01, verbosity =0)

    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    casecount = 0
    rowcounter = 0
    running_case = 0
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    
    # Sliding window for training setting
    window_size = 100
    test_size = 30
    training_windows = sliding_window.training_window(window_size,test_size)
    training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier, 0]
    
    dataset = stream.iter_csv(
            dataset_loc
            )
    totallength = len(list(dataset))


    dataset = stream.iter_csv(
                dataset_loc,
                target='outcome'
                )
    enctype = 'Index-base'
    
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0

    for x,y in tqdm(dataset):
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            case_length = len(streaming_db[caseid])
            if case_length >= maximum_prefix:
                if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                    feature_matrix['prefix_%s'%(maximum_prefix)]=set()

                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    x = streaming_db[caseid][maximum_prefix-1]
                    training_windows.update_window(x)
                    update_test_cases(streaming_db[caseid], test_cases)

                    if training_windows.retraining_count == test_size:
                        x_training = pd.DataFrame.from_dict([i.encoded for i in training_windows.container])
                        for i in x_training.columns.values: x_training[i] = x_training[i].fillna(0)
                        feature_matrix['prefix_%s'%(maximum_prefix)] = x_training.columns.values
                        y_training = [i.true_label for i in training_windows.container]
                        training_models['prefix_%s'%(maximum_prefix)][0].fit(x_training, y_training)
                        training_models['prefix_%s'%(maximum_prefix)][1] +=1
                        training_windows.reset_retraining_count()

                    '''
                    Evaluate current model with test cases
                    '''
                    prediction_result[maximum_prefix][casecount] = {}
                    y_truelist = []
                    y_predlist = []
                    for case in test_cases:
                        if len(case) >= maximum_prefix:
                            x = case[maximum_prefix-1]
                            model = training_models['prefix_%s'%(x.prefix_length)][0]
                            current_event = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                            current_event = pd.Series(current_event).to_frame().T
                            if training_models['prefix_%s'%(maximum_prefix)][1] !=0:
                                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba(current_event)
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                    prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                    prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
                    if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                        cdhappend[maximum_prefix] = casecount
    for evaluation_method in ['Accuracy', 'F1', 'ROCAUC']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob_ = prediction_result[prefix][update]['y_pred']
            prediction_class = training_models['prefix_%s'%(maximum_prefix)][0].classes_
            if len(prediction_class) == 1: prediction_class = ['False', 'True']
            y_pred_prob = []
            for i in y_pred_prob_:
                y_pred_prob.append({prediction_class[0]: i[0][0], prediction_class[1]: i[0][1]})
            y_pred = [max(x, key=x.get) for x in y_pred_prob]
            update_count.append(update)

            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true, y_pred, average='macro')
        #             warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                y_pred_prob = np.array([[x['False'], x['True']] for x in y_pred_prob])
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:, 1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')
        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)

        if os.path.exists('../img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('../img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))
        if os.path.exists('../result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('../result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))


        with open('../result/%s/%s/Finished cases/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('../img/%s/%s/Finished cases/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Resource': 'resource', 'Start Timestamp': 'ts'}, 'categorical_attrs': ['activity', 'resource'], 'maximum_prefix': 15}
1


23941it [02:58, 133.84it/s] 


2


23941it [02:49, 141.58it/s] 


3


...

23941it [03:28, 114.82it/s] 


11


23941it [03:11, 125.21it/s] 


12


15035it [01:02, 71.51it/s]  

In [9]:
training_models['prefix_%s'%(x.prefix_length)][1]

0